# Introduction to ECoG and EEG

## Overview
Our first class is going to cover the background of Electroencephalography (EEG) and Electrocorticography (ECoG). These are the kinds of data we'll use in the first half of this class. Below is a list of topics that we'll focus on.

## Goals for today
* Load some raw data into python
* Simple visualizations to understand the kind of data we have
* Learn some things about the data by scanning the raw values
* Introduction to the MNE library for electrophysiology

---

# Background
See the Powerpoint for this week for a quick intro to ECoG and EEG. It has some background on the history of these methods, what kinds of signals they measure, and what kinds of things we'll try to do with them.


# Visualizing electrophysiology data in python

## What is a timeseries?
In neuroscience, the most common form of data is called a *timeseries*. This essentially means that we have a sensor that collects information about some signal in the brain as it changes across time.

A timeseries can be anything from stock market fluctuations, to the changes in score during a basketball game, to electrical fluctuations in the brain.

In this course, we discuss two types of timeseries from the human brain:

1. Voltage on the brain surface, recorded from electrodes.
2. A measure of blood flow in the cortex of the brain, recorded with an fMRI machine.



In [ ]:
import os
import mne
from matplotlib import pyplot as plt
import numpy as np
import os.path as op
from glob import glob
from warnings import simplefilter
simplefilter('ignore')
import neurods
import sys

In [ ]:
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'

In [ ]:
%matplotlib inline

## A sample timeseries
First, let's load a sample EEG timeseries so we can see what it looks like.

In [ ]:
path_data = glob(neurods.io.data_list['eeg'] + '/subject8/session1/*-raw.fif')[0]
path_events = glob(neurods.io.data_list['eeg'] + '/subject8/session1/*-eve*')[0]
raw = mne.io.Raw(path_data, preload=True)

# Drop mastoid channels
chs_drop = ['MA1', 'MA2']
raw.drop_channels(chs_drop)

In [ ]:
# The eeg data in this object is accessible at `raw._data`:
raw._data

## Breakout session
> What does this data look like? explore it in as many ways as you can think of!
> Hint: Try to plot individual EEG channels, and also multiple channels!

***NOTE***: Don't forget to that the plt.plot() function plots the FIRST dimension of a 2D array as lines; the SECOND dimension determines the number of lines. So if you call `plt.plot(A)`, for which `A` is an array of shape (32, 100000), then you will kill your kernel because you are trying to plot 100000 lines! **This is why we often transpose the arrays (`.T`) within the plot commands below - to switch which dimension is being plotted!**

In [ ]:
### STUDENT ANSWER
from scipy.stats import zscore

# Plot a single channel
plt.figure()
plt.plot(raw._data[0, :10000].T)

# Plot multiple channels
plt.figure()
plt.plot(raw._data[:10, :10000].T)

# Plot a histogram of all the data
plt.figure()
plt.hist(raw._data.flatten(), bins=50)
plt.title("All data histogram")

# Plot histograms of a few individual channels
plt.figure()
for i in range(3):
    plt.hist(raw._data[i], bins = 50)
plt.title('Histograms for three channels')

# Plot the data using imshow
plt.figure()
plt.imshow(raw._data)
plt.title('All data as an image\n (something is up with the last time values!)')

What did you notice about the activity in these channels? Is it similar to what we saw in fMRI?

How could you manipulate the data (add something, multiply by something, subtract something) to make a more sensible / informative plot?

In [ ]:
### STUDENT ANSWER
# Plot a z-scored image of the data (zscore on last axis, time)
data_z = zscore(raw._data, axis=1)
plt.figure()
plt.imshow(data_z)
plt.title("All data, zscored over time, as an image")

# Slice the data a little to remove outlying values at end, THEN zscore
data_z = zscore(raw._data[:,:10000], axis=1)
plt.figure()
plt.imshow(data_z)
plt.title("All data, clipped & zscored over time as an image")
plt.colorbar()

# Same, with line plots:
plt.figure()
plt.plot(zscore(raw._data[:10, :1000].T)) 
plt.xlabel('Samples')
plt.ylabel('Data Values')

Remember that it's always important to take a look at the raw data first, and to make sure you've standardized / cleaned up the data if there are any obvious problems at the very beginning.

## Using MNE to represent our data
Because a timeseries has a specific structure (signals varying over time), there are a number of packages designed to handle this particular kind of data. One of the best python packages for neuroscience is called `MNE-python`. Check out their website here: http://martinos.org/mne/stable/index.html

MNE has code that can handle datasets like the one we've used above. The main class we'll focus on are `Raw` objects. These are similar to the `.nii` files that we're been working with in fMRI, but they represent EEG data instead.

`Raw` data objects are simple a way of keeping the data (which is of shape `n_channels x n_times`) along with some metadata that gives extra information about our data. Note that the convention here is different from fMRI data: time is the LAST dimension instead of the FIRST! (Confusing ,yes; so it goes.)

For example, any object that represents data in `MNE-python` will have an `info` attribute. This is a collection of information about the data at hand. It includes information like:

* `info['sfreq']` - The sampling frequency of the data
* `info['ch_names']` - The names of channels in the data
* `info['bads']` - A list of "bad" channels
* `info['chs']` - A more complex collection of channel information, such as xyz location.

There are many other pieces of information related to things that are important in EEG analysis, most of which we won't go into.

In [ ]:
# Here's a list of all the things in an Info object
raw.info

We can also always access the numpy array of data by accessing the `._data` attribute. **Note** This only works if the data has been *preloaded* (MNE sometimes holds off on loading the data from disk so that it doesn't use up memory, just like we saw with the nibabel package for fMRI data).

In [ ]:
# Make sure the data is loaded
raw.load_data()

In [ ]:
print(raw._data.shape)
print(raw._data)

### Sampling Frequency

The time values for this signal tell us how much time passes between observations of the signal. It's a crucial piece of information in order to do many analyses in neuroscience.

The **sampling frequency** (or sampling rate) is the number of samples we record *per second*. We can calculate this by this equation:

$$sampling\_frequency = samples\_per\_second = \frac{n\_samples}{1\_second} = \frac{1\_sample}{seconds\_between\_samples} = \frac{1}{signal\_period}$$

We know the sampling frequency of this dataset (it's stored in raw.info) How could we use it to calculate the time dimension?

In [ ]:
### STUDENT ANSWER
time = np.arange(raw.n_times) / raw.info['sfreq']

fig, ax = plt.subplots()
_ = ax.plot(time[:1000], raw._data[0, :1000].T)
plt.xlabel("Time (seconds)")
plt.ylabel("EEG activity (raw voltage)")

## Plotting raw data with MNE
MNE has a lot of plotting functionality, which is one of the most important things to do in data analysis. Below we'll explore the basic ways in which we can visualize our data.

In [ ]:
# To use this, we'll need to activate "interactive" mode
# This lets us keep plots interactive in the notebook
# NOTE: as we saw in class, there are some bugs associated with this!
%matplotlib notebook

First, we'll plot the raw data. This will give us an output of traces, one for each channel. The x-axis is time (in seconds). What we are looking at is the electrical activity (voltage) recorded at the scalp, and generated by neural activity.

In [ ]:
# Putting `_` before the equals sign tells Python to throw away the output
# Otherwise, it will make 2 plots.
raw.crop(0, 40)
raw._data = raw._data - raw._data.mean(-1)[:, np.newaxis]
_ = raw.plot(scalings='auto')

### Breakout session
Scan through this dataset using the `raw.plot` function.

* Take note of the shapes and sizes of the signal in each electrode.
* In addition, see if you notice any points in time where the nature of the signal changes.

## Smoothing our data
You may notice that the raw signal seems to be quite noisy. Some electrodes have activity that is jumping around quite a lot, there are also some electrodes that seem to "drift" over time, meaning their average value slowly gets larger or smaller.

Let's see an example of one plot of data below...

In [ ]:
%matplotlib inline

In [ ]:
raw_filt = mne.io.Raw(path_data, preload=True)

In [ ]:
# First, we'll plot a single channel
fig, ax = plt.subplots()
ch_plt = 2
ax.plot(raw_filt._data[ch_plt, :6000].T)

Notice how there are different kinds of activity here. There are some more slow-changing parts (e.g. the bumps that occur over 100 samples. There are also some really "jittery" parts of the signal that quickly bounce up and down.

Neuroscientists often assume that the jittery parts of our data correspond to noise in the system. This kind of activity has a "high frequency", meaning that it changes very quickly.

To emphasize either the slow-moving or fast-moving parts of our signal, we can *filter* it. This means removing the parts of the signal that we aren't interested in. For example, if we want to remove the fast-changing jittery stuff, we can filter out the higher frequency activity in the signal. This is called a *low pass filter*.

In [ ]:
fmin, fmax = None, 40
raw_filt.filter(fmin, fmax)
fig, ax = plt.subplots()
ax.plot(raw_filt._data[ch_plt, :6000].T)

### Breakout session
We have mentioned "drift" a few times thus far. This occurs when the signal changes very slowly over time. One way to think of this kind of activity is that it is a very low-frequency component of the signal (AKA, it changes slowly). See if you can use the `.filter` method on our raw object to remove this drift in the signal.

In [ ]:
### STUDENT ANSWER
raw_filt.filter(2, None)
fig, ax = plt.subplots()
ax.plot(raw_filt._data[ch_plt, :6000].T)

Below we can see how several kinds of filters have different effects on our data. We can remove the high-frequency parts of the signal as we did above (called low-pass filtering or smoothing), we can also remove the low-frequency parts of the signal as we did in the breakout session (called high-pass filtering). We can even isolate a part of the signal that is oscillating with a particular frequency (called a band-pass filter).

In [ ]:
# Different kinds of filters produce different results:
raw_filt = mne.io.Raw(path_data, preload=True).crop(0, 20)

# These are filters we'll use on the data
filters = [(1, None), (20, None), (50, None),
           (None, 2), (None, 20), (None, 50),
           (2, 3), (19, 21), (50, 52)]

# Plot our original signal
fig, ax = plt.subplots()
tmin, tmax = 2, 5 # min, max time in seconds
ax.plot(raw_filt.copy().crop(tmin, tmax)._data[ch_plt])
ax.set(title='Original Signal')

# Make a plot for each type of filtering
fig, axs = plt.subplots(3, 3, figsize=(15, 10), sharex=True, sharey=True)
for ax, (fmin, fmax) in zip(axs.ravel(), filters):
    raw_plt = raw_filt.copy()
    raw_plt = raw_plt.filter(fmin, fmax, verbose=False)
    raw_plt = raw_plt.crop(tmin, tmax)
    raw_plt._data = raw_plt._data - raw_plt._data.mean(-1)[:, np.newaxis]

    ax.plot(raw_plt._data[ch_plt])
    ax.set(title='Filter: {} to {}'.format(fmin, fmax))

In [ ]:
# Define a general function to load data, as we did with fMRI
def load_eeg_data(fname, ref_electrode_idx=(6,23), verbose=False,
                  filter_cutoffs=(1,12), return_mne=True):
    """Load EEG data"""
    
    
    raw = mne.io.Raw(fname, preload=True)
    # Remove mastoid channels
    raw._data = raw._data[:-2]
    # Reference signal to temporal lobe electrodes
    if ref_electrode_idx is not None:
        ref_electrodes = raw._data[list(ref_electrode_idx)]
        raw._data -= ref_electrodes.mean(0)
    # Smooth the data
    if filter_cutoffs is not None:
        fmin, fmax = filter_cutoffs # Set cutoffs for low and high frequency in data
        raw._data = mne.filter.filter_data(raw._data, raw.info['sfreq'], fmin, fmax, verbose=verbose)
    if return_mne:
        return raw
    else:
        return raw.times, raw._data

In [ ]:
subject, session, run = 8, 1, 0
fdir = os.path.join(neurods.io.data_list['eeg'], 'subject{}/session{}/'.format(subject, session))
files = sorted(glob(os.path.join(fdir, '*fif')))
fname = files[run]
times, data = load_eeg_data(fname, return_mne=False)
print(data.shape)

In [ ]:
_ = plt.plot(data[:10,:1000].T)

## Plotting sensor locations
Now that we've learned a little bit about the activity itself, there's one more thing we need to know: where these channels are located.

Patterns of recorded neural activity wouldn't be very useful without knowing where that activity was generated. In EEG our sensors sit on the scalp of a person's head. The activity we've been plotting represents electrical activity occurring inside their brains, that makes its way through their skull and ultimately onto their scalp.

MNE has simple functions for plotting the location of sensors with EEG. We can see this below:

In [ ]:
%matplotlib inline

In [ ]:
# A Montage represents the location of each electrode on the scale
mon = mne.channels.read_montage('standard_1005', raw.ch_names)
raw.set_montage(mon)

In [ ]:
# We can look at the layout of channels on the brain.
# Almost all EEG setups have the same general structure.
_ = raw.plot_sensors(show_names=True)

In [ ]:
%matplotlib notebook

In [ ]:
# We can also plot in 3d
# This addresses the afore-mentioned bug!
fig = raw.plot_sensors('3d')

In [ ]:
# (now display the object that the last cell created) # THIS IS A HACKY WORKAROUND FOR A BUG
fig

### Question
* How given the electrode location plots above, how do you think neuroscientists made this movie:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('xmf3YC8jzWc')

# Simple event slicing
The last thing we'll look at is how to represent event information. Knowing about brain activity isn't very useful unless you also knew what was happening in the world. For this reason, neuroscientists often record events that correspond to a particular kind of stimulus being shown to a person.

Below we'll show a simple way of representing events in python:

In [ ]:
%matplotlib inline
events = mne.read_events(path_events)
events[:5]

### Breakout session
* Use matplotlib to plot the first and the third column of the array above. Plot the first 100 samples.
* Now try the same thing, but for histograms (hint, use at least 20 bins in the histogram).
* What do you think each of these two columns represents?

In [ ]:
### STUDENT ANSWER
for col in [0, 2]:
    fig, axs = plt.subplots(1, 2)
    axs[0].plot(events[:100, col])

    axs[1].hist(events[:100, col], bins=20)

We can also use MNE to plot the event array itself. This makes it very easy to see what is going on...

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
_ = mne.viz.plot_events(events, sfreq=raw.info['sfreq'], axes=ax)

Let's visualize our EEG data along with these events

In [ ]:
%matplotlib notebook

In [ ]:
# Same hacky work-around for bug
fig = raw.plot(events=events, scalings='auto')

In [ ]:
fig

# Wrapup part 1
In this lecture we covered a couple of basic topics:
* How to visualize the raw data
* How to manipulate the raw data to highlight particular features of the data
* How to show where each channel lies on the brain
* How to note the time onsets of events in our data

These are all the basic tools that we need in order to start figuring out what the brain is doing in response to experiments that we run. The next section will cover one such application.

# Computing event related averages in EEG
Last lecture (and for homework), we computed event-related averages of fMRI data. To do this, we selected all data points after a set of event onset indices. Here, we will do the same with EEG data.

In [ ]:
times, data = load_eeg_data(fname, return_mne=False)

In [ ]:
# Select the onsets of a particular condition using a logical index
cond_1 = events[:,2]==1
onset_indices = events[cond_1,0]
print(onset_indices)

# Breakout session
> Modify the following cell to select the ***time*** period from 0.1 seconds BEFORE the event onset to 0.6 seconds AFTER the event onset

NOTE: This is now your homework!

In [ ]:
epochs = []
duration = 10 # This value is wrong, even for the code as written! What should this be? 
              # How should you modify the code to select a duration in time?
for o in onset_indices:
    data_slice = data[:, o:(o+duration)]
    # This if loop is here due to a small problem with the data... 
    if np.any(data_slice):
        epochs.append(data_slice)
epochs = np.array(epochs)
print(epochs.shape)